In [ ]:
import json, os
import numpy as np,  pandas as pd
from salk_toolkit.io import read_annotated_data

## Example: master.csv

In [ ]:
survey_meta = {
    "file": "master.csv",
    "read_opts": { "sep": "," },
    "preprocessing": "df.loc[df.D4 == 'Tartu linn', 'T6'] = 'Tartu linn'; df.loc[df.D4 == 'Tallinn (L)', 'T6'] = df.D4b",
    "constants": {
        "districts": {
            "Tallinn": "Tallinn",
            "Harjumaa (välja arvatud Tallinn)": "Harju- ja Raplamaa",
            "Harjumaa": "Harju- ja Raplamaa",
            "Läänemaa": "Hiiu-, Lääne- ja Saaremaa",
            "Hiiumaa": "Hiiu-, Lääne- ja Saaremaa",
            "Pärnumaa": "Pärnumaa",
            "Võrumaa": "Võru-, Valga- ja Põlvamaa",
            "Ida-Virumaa": "Ida-Virumaa",
            "Tartumaa": "Jõgeva- ja Tartumaa",
            "Järvamaa": "Järva- ja Viljandimaa",
            "Põlvamaa": "Võru-, Valga- ja Põlvamaa",
            "Valgamaa": "Võru-, Valga- ja Põlvamaa",
            "Viljandimaa": "Järva- ja Viljandimaa",
            "Raplamaa": "Harju- ja Raplamaa",
            "Jõgevamaa": "Jõgeva- ja Tartumaa",
            "Lääne-Virumaa": "Lääne-Virumaa",
            "Saaremaa": "Hiiu-, Lääne- ja Saaremaa",
            "Haabersti": "Haabersti, Põhja-Tallinn ja Kristiine",
            "Põhja-Tallinn": "Haabersti, Põhja-Tallinn ja Kristiine",
            "Kristiine": "Haabersti, Põhja-Tallinn ja Kristiine",
            "Kesklinn": "Kesklinn, Lasnamäe ja Pirita",
            "Lasnamäe": "Kesklinn, Lasnamäe ja Pirita",
            "Pirita": "Kesklinn, Lasnamäe ja Pirita",
            "Mustamäe": "Mustamäe ja Nõmme",
            "Nõmme": "Mustamäe ja Nõmme",
            "Tartu linn": "Tartu linn"
        },
        "party_colors": {
            "EKRE": "#8B4513",
            "Eesti 200": "#31758A",
            "Isamaa": "#009BDF",
            "Keskerakond": "#007557",
            "Reformierakond": "#FFE200",
            "Rohelised": "#88AF47",
            "SDE": "#E10600",
            "Parempoolsed": "orange",
            "No party": "grey",
            "Hard to say": "lightgrey",
            "Other": "lightgrey"
        }
    },
    "structure": [{
        "name": "meta",
        "columns": [
            ["wave", "laine"],
            ["t", "laine", { "transform": "s.replace(dict(((df.groupby('laine')['date'].apply(lambda s: pd.to_datetime(s).median() ) - pd.to_datetime('2023-03-05')).dt.days/30).items()))"    }],
            ["date", { "transform": "pd.to_datetime(s)" }],
            ["methods", { "categories": "infer" , 'translate': {'WISE': 'Cati'} }]
        ]
    }, {
        "name": "demographics",
        "columns": [
            "age", 
            ["age_group", { "categories": ["16-24", "25-34", "35-44", "45-54", "55-64", "65-74", "75+"],    "ordered":     True    }],
            ["education", "T8", {
                "categories": ["Basic education", "Secondary education", "Higher education"], "ordered":     True,
                "translate": { "Põhiharidus": "Basic education", "Keskharidus": "Secondary education", "Kõrgharidus": "Higher education" }
            }],
            ["gender", {
                "categories": ["Male", "Female"],
                "translate": {    "Mees": "Male",    "Naine": "Female" }
            }],
            ["nationality", "T3c", {
                "categories": ["Estonian", "Other"],
                "translate": { "Eestlane": "Estonian", "Muu": "Other", "Venelane": "Other" }
            }],
            ["electoral_district", "T6", { "categories": "infer", "translate": "districts" }],
            ["unit", "T6", { "categories": "infer", "translate": { "Harjumaa (välja arvatud Tallinn)": "Harjumaa" }    }],
            ["citizen", "K0", {    "transform": "s=='Jah'" }]
        ]
    }, {
        "name": "wedge",
        "scale": {
            "categories": ["Totally disagree", "Rather disagree", "Hard to say", "Rather agree", "Totally agree"],    "ordered":     True,
            "translate": { "Ei oska öelda": "Hard to say", "Täiesti nõus": "Totally agree", "Üldse ei nõustu": "Totally disagree", "Pigem nõus": "Rather agree", "Pigem ei ole nõus": "Rather disagree" }
        },
        "columns": [
            ["e-valimised", "Q1r1"],
            ["rahvahääletus", "Q1r2"],
            ["sisseränne", "Q1r3"],
            ["Euroopa Liit", "Q1r5"],
            ["eestikeelne haridus", "Q1r6"],
            ["kliimamuutus", "Q1r7"],
            ["metsaraie", "Q1r8"],
            ["kooselu", "Q1r10"],
            ["abort", "Q1r12"],
            ["sõnavabadus", "Q1r13"],
            ["jõukamad", "Q1r14"],
            ["koroonapiirangud", "Q1r15"],
            ["NATO", "Q1r16"],
            ["usaldus", "Q1r17"]
        ]
    }, {
        "name": "trust",
        "scale": {
            "categories": ["Don't trust at all", "Rather not trust", "Hard to say", "Rather trust", "Trust completely"], "ordered":     True,
            "translate": { "Ei oska öelda": "Hard to say", "Pigem ei usalda": "Rather not trust", "Usaldan täiesti": "Trust completely", "Pigem usaldan": "Rather trust", "Üldse ei usalda": "Don't trust at all" }
        },
        "columns": [
            ["valitsus", "Q3r1"],
            ["riigikogu", "Q3r2"],
            ["kohtusüsteem", "Q3r3"],
            ["ajakirjandus", "Q3r4"]
        ]
    }, {
        "name": "voting_behavior",
        "columns": [
            ["party_preference", "Q4r1", {
                "categories": "infer",
                "translate": { "Eesti Keskerakond": "Keskerakond", "Eesti Reformierakond": "Reformierakond", "Eesti Konservatiivne Rahvaerakond (EKRE)": "EKRE", "ISAMAA Erakond": "Isamaa", "Erakond Eesti 200": "Eesti 200", "Sotsiaaldemokraatlik Erakond": "SDE", "Erakond Eestimaa Rohelised": "Rohelised", "Erakond Parempoolsed": "Parempoolsed", "Mitte ükski erakond": "No party", "Ei oska öelda": "Hard to say"  },
                "colors": "party_colors"
            }],
            ["voting_intent", "Q5", {
                "categories": ["No", "No opinion", "Yes"], "ordered":     True,
                "translate": { "Ei": "No", "Ei soovi avaldada": "No opinion", "Jah": "Yes" }
            }]
        ]
    }, {
        "name": "compass",
        "columns": ["Keskerakond", "EKRE", "Reformierakond", "Isamaa", "SDE", "Rohelised", "Eesti 200", "Parempoolsed"],
        "colors": "party_colors"
    }, {
        "name": "extras",
        "columns": [
            ["income", "T10", {
                "categories": ["No opinion", "<500 €", "501-1000 €", "1001-1500 €", "1501-2000 €", ">2000 €"], "ordered":     True,
                "translate": { "Ei soovi avaldada": "No opinion", "Kuni 500 €": "<500 €", "Üle 2000 €": ">2000 €" }
            }],
            ["children", "T5", {
                "categories": ["No opinion", "0", "1", "2", "3+"],    "ordered":     True,
                "translate": {    "Ei soovi avaldada": "No opinion", "Ei ole lapsi": "0", "1 laps": "1",    "2 last": "2", "3 või rohkem last": "3+" }
            }],
            ["occupation", "T9", {
                "categories": "infer",
                "translate": { "Õpilane või üliõpilane": "Student",    "Tööline": "Worker", "Juht": "Manager",    "Töötu, tööd otsiv": "Unemployed, looking for work", "Spetsialist": "Specialist", "Ettevõtja või füüsilisest isikust ettevõtja (FIE)": "Entrepreneur or self-employed person (FIE)", "Kodune või mittetöötav": "Domestic or non-working", "Pensionär": "Pensioner" }
            }]
        ]
    }]
}
#with open('data/master_meta.json','w',encoding='utf-8') as f:
#    json.dump(survey_meta,f,ensure_ascii=False)

In [ ]:
#raw_data = pd.read_csv("./data/master.csv", index_col=0, engine="python")
data = read_annotated_data(meta=survey_meta, data_file='./data/master_fragment.csv')
data

Column Q3r3 not found


,wave,t,date,methods,age,age_group,education,gender,nationality,electoral_district,...,EKRE,Reformierakond,Isamaa,SDE,Rohelised,Eesti 200,Parempoolsed,income,children,occupation
0,18,-4.500000,2022-10-11 15:02:00,Cati,70.0,65-74,Secondary education,Female,Estonian,"Võru-, Valga- ja Põlvamaa",...,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,<500 €,1,Pensioner
1,20,-2.900000,2022-12-14 14:37:00,Web,43.0,35-44,Higher education,Female,Other,Mustamäe ja Nõmme,...,-1.0,-4.0,-3.0,0.0,0.0,0.0,0.0,>2000 €,2,Specialist
2,14,-8.866667,2022-06-22 08:51:00,Web,25.0,25-34,Higher education,Female,Estonian,Harju- ja Raplamaa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,501-1000 €,1,Specialist
3,1,-23.066667,2021-04-16 11:58:00,Cati,26.0,25-34,Secondary education,Female,Estonian,Harju- ja Raplamaa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,501-1000 €,0,Worker
4,21,-1.933333,2023-01-12 18:27:38,Web,48.0,45-54,Higher education,Female,Other,Harju- ja Raplamaa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,>2000 €,2,Worker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,-21.566667,2021-05-29 11:28:00,Cati,30.0,25-34,Secondary education,Male,Estonian,Tartu linn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,501-1000 €,0,Worker
96,2,-21.566667,2021-05-26 11:39:00,Cati,77.0,75+,Higher education,Female,Other,Harju- ja Raplamaa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,501-1000 €,2,Pensioner
97,10,-13.133333,2022-02-04 14:07:00,Cati,74.0,65-74,Secondary education,Female,Other,Harju- ja Raplamaa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,501-1000 €,2,Pensioner
98,21,-1.933333,2023-01-07 15:11:54,Cati,56.0,55-64,Secondary education,Female,Estonian,Järva- ja Viljandimaa,...,4.0,-5.0,-3.0,-5.0,0.0,-3.0,-2.0,No opinion,3+,Domestic or non-working
